# [MLAC] 23개의 공격 하위(세부) 타입 분류

### 데이터 셋 불러오기

In [2]:
import pandas as pd

ConcatedCICI = pd.read_csv('/home/irteam/wendyunji-dcloud-dir/wendyunji/Dataset/MLAC-dataset/encoded_ConcatedCICI.csv')
ConcatedUNSW = pd.read_csv('/home/irteam/wendyunji-dcloud-dir/wendyunji/Dataset/MLAC-dataset/encoded_ConcatedUNSW.csv')

In [3]:
# Benign은 제거
ConcatedCICI.drop(ConcatedCICI[ConcatedCICI['label']==0].index, inplace=True)
ConcatedUNSW.drop(ConcatedUNSW[ConcatedUNSW['label']==0].index, inplace=True)

# label 칼럼 제거
del ConcatedCICI['label']
del ConcatedUNSW['label']

### Attack Category 확인 및 실제 값으로 매핑

In [4]:
print(ConcatedCICI['attack_category'].value_counts())

7     231073
16    158930
4     128027
10     59131
12     25621
5      22493
13     18100
17     15203
6      10293
11      7938
18      5897
9       5796
8       5499
2       4070
3       1966
1       1695
20      1507
19       865
22       652
23       105
15        36
21        21
14        11
Name: attack_category, dtype: int64


In [5]:
print(ConcatedUNSW['attack_category'].value_counts())

13    215481
7     161097
16    158935
4      96027
10     44525
12     24246
5      16353
17     13987
8       8835
6       8780
9       8706
11      3991
18      2984
1       2677
2       2329
19      1511
20      1365
3       1230
22       648
23       174
15        59
14        21
21        12
Name: attack_category, dtype: int64


In [6]:
# 숫자를 키로, 공격명을 값으로 가지는 딕셔너리 생성
attackname = [
    'Dos Hulk',
    'PortScan',
    'DDoS',
    'DoS GoldenEye',
    'FTP-Patator',
    'SSH-Patator',
    'DoS slowloris',
    'DoS Slowhttptest',
    'Bot',
    'Web Attack – Brute Force',
    'Web Attack – XSS',
    'Infiltration',
    'Web Attack – Sql Injection',
    'Heartbleed',
    'Generic',
    'Exploits',
    'Fuzzers',
    'DoS',
    'Reconnaissance',
    'Analysis',
    'Backdoor',
    'Shellcode',
    'Worms',
]

attackname.sort()
attackdict = {}
for i in range(len(attackname)):
    attackdict[i+1] = attackname[i]
    
print(attackdict)

{1: 'Analysis', 2: 'Backdoor', 3: 'Bot', 4: 'DDoS', 5: 'DoS', 6: 'DoS GoldenEye', 7: 'DoS Slowhttptest', 8: 'DoS slowloris', 9: 'Dos Hulk', 10: 'Exploits', 11: 'FTP-Patator', 12: 'Fuzzers', 13: 'Generic', 14: 'Heartbleed', 15: 'Infiltration', 16: 'PortScan', 17: 'Reconnaissance', 18: 'SSH-Patator', 19: 'Shellcode', 20: 'Web Attack – Brute Force', 21: 'Web Attack – Sql Injection', 22: 'Web Attack – XSS', 23: 'Worms'}


### 전체 정확도 얻기

In [7]:
import classification_util as Cutils

In [8]:
files = ['ConcatedCICI', 'ConcatedUNSW']
datas = [ConcatedCICI, ConcatedUNSW]

In [9]:
def Cpipeline(file, data):
    X_train, X_test, y_train, y_test = Cutils.TrainTestSplit(data)
    MLmodels = Cutils.getModels()
    Cutils.MultiClassification(file, MLmodels, X_train, X_test, y_train, y_test)

In [11]:
for i in range(len(files)):
    Cpipeline(files[i],datas[i])

Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
RF	0.871	0.871	0.871	0.871	0.404	0.381	0.527	0.846	0.871	0.870	2.15 secs
DT	0.861	0.861	0.861	0.861	0.325	0.299	0.437	0.831	0.861	0.829	8.36 secs
NB	0.586	0.586	0.586	0.586	0.377	0.585	0.458	0.635	0.586	0.874	2.32 secs
LDA	0.861	0.861	0.861	0.861	0.569	0.640	0.605	0.865	0.861	0.909	6.28 secs
QDA	0.827	0.827	0.827	0.827	0.477	0.568	0.562	0.833	0.827	0.933	4.76 secs
LR	0.907	0.907	0.907	0.907	0.556	0.554	0.613	0.891	0.907	0.918	69.11 secs
ABoost	0.638	0.638	0.638	0.638	0.151	0.178	0.145	0.518	0.638	0.458	140.98 secs
k-NN	0.903	0.903	0.903	0.903	0.668	0.652	0.765	0.900	0.903	0.899	30.61 secs
MLP	0.916	0.916	0.916	0.916	0.624	0.622	0.757	0.902	0.916	0.929	518.72 secs
SVM	0.909	0.909	0.909	0.909	0.616	0.619	0.664	0.893	0.909	0.897	302.74 secs
Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
RF	0.812	0.812	0.812	0.812	0.224	0.217

### 공격 별 정확도 얻기

In [ ]:
import evaluation_utils as Eutils

# 함수를 실행하기 위한 정보
files = ['ConcatedCICI', 'ConcatedUNSW']
modelnames = ['RF','DT','NB','LDA','QDA','LR','ABoost','k-NN','MLP','SVM']

for i in range(len(files)):
    #fprfile = open('/home/irteam/wendyunji-dcloud-dir/wendyunji/MLAC/Classification/Evaluation/Binary/FPR/'+file[i]+'.txt','w')
    f = open('/home/irteam/wendyunji-dcloud-dir/wendyunji/2023-1/MLAC/Classification/result/230330/attackpereval_'+files[i]+'.txt','a')
    for j in range(len(modelnames)):
        f.write(str(modelnames[j])+'\t')
        txt = '/home/irteam/wendyunji-dcloud-dir/wendyunji/2023-1/MLAC/Classification/result/230330/matrix/'+files[i]+'_'+modelnames[j]+'.txt'
        matrix = Eutils.CMtoList(txt)
        metrics = Eutils.MultiMetric(f, matrix)
        # print(metrics)
        #fprfile.write(modelnames[j]+'\t'+str(fpr)+'\n')

### Best Model의 Feature importance 구하기

Best Model이 MLP로 선정됨에 따라 모델의 Feature importance를 구하기 위한 방법으로는 permutation_importance 함수를 사용하여 각 feature의 permutation importance를 계산할 수 있습니다. 이 방법은 특정 feature를 무작위로 섞어서 모델의 성능 변화를 측정하고, 이를 통해 각 feature의 중요도를 계산합니다. 이 방법은 non-linear 모델에서도 적용 가능하며, feature 간 상호작용에 대한 정보도 제공합니다.
 (https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html )

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.datasets import make_classification
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
def mlpFI(data):
    clf = MLPClassifier()
    X_train, X_test, y_train, y_test = Cutils.TrainTestSplit(data)
    clf.fit(X_train, y_train)
    result = permutation_importance(
        clf,
        X_train,
        Y_train,
        scoring='f1',
        
    )

In [ ]:
perm = PermutationImportance(model, scoring = "f1", random_state = 42).fit(X_val, y_val)
eli5.show_weights(perm, top = 80, feature_names = X_val.columns.tolist())